# Day ? Random Forest for Amazon River classification

In [14]:
from pathlib import Path
import os
from sentinelsat import SentinelAPI
from osgeo import gdal

In [15]:
path = Path('../data/rivers_amazon_data/random_forest_test/')

In [21]:
# ! ls {path}

In [19]:
# username = os.environ.get('COPERNICUS_USERNAME')
# password = os.environ.get('COPERNICUS_PASSWORD')
# api = SentinelAPI(user=username, password=password)
# api.download('dcab558f-9b8d-4935-99d5-881144e17348', path)
# product_id = (str(list(path.glob('*'))).split('/')[-1])[:-7];
# ! unzip {path}/{product_id + '.zip'} -d {path}

# parts = [product_id + '.SAFE', 
#         'GRANULE',
#         'L1C_T21MYS_A016080_20180721T140053',
#         'IMG_DATA']

# for i in [2,3,4,8]:
#     image_band = 'T21MYS_20180721T140051_B0{}.jp2'.format(i)
#     filepath = str(path.joinpath(*parts).joinpath(image_band))
#     out_location = str(path) + ('/ndwi_B0{}'.format(i)) + '.tif'
#     src = gdal.Open(filepath)
#     gdal.Translate(out_location, src)

In [18]:
band1_fn = str(path.joinpath('ndwi_B04.tif')) #Red
band2_fn = str(path.joinpath('ndwi_B03.tif')) #Green
band3_fn = str(path.joinpath('ndwi_B02.tif')) #Blue
band4_fn = str(path.joinpath('ndwi_B08.tif')) #NIR

in_ds = gdal.Open(band1_fn)
in_band = in_ds.GetRasterBand(1)
gtiff_driver = gdal.GetDriverByName('GTiff')
out_path = str(path) + '/10m_4band_day18.tif'
out_ds = gtiff_driver.Create(out_path, in_band.XSize,
                            in_band.YSize, 4, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform(in_ds.GetGeoTransform())

in_data = in_band.ReadAsArray()
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(in_data)

out_ds.GetRasterBand(2).WriteArray(gdal.Open(band2_fn).ReadAsArray())
out_ds.GetRasterBand(3).WriteArray(gdal.Open(band3_fn).ReadAsArray())
out_ds.GetRasterBand(4).WriteArray(gdal.Open(band4_fn).ReadAsArray())

out_ds.FlushCache()
for i in range(1,5):
    out_ds.GetRasterBand(i).ComputeStatistics(False)
out_ds.BuildOverviews('average', [2,3,8,16,32])

del out_ds

#source: Geoprocessing with Python by Chris Garrard

### Getting training data

OSM data fro Jochen and Christoph on [openstreetmapdata](http://openstreetmapdata.com/data) is low resolution and does not cover smaller rivers. 